In [15]:
import pandas as pd
import itertools

account_booking_test = pd.read_csv("account_booking_test.csv")
account_booking_train = pd.read_csv("account_booking_train.csv")
external_parties_test = pd.read_csv("external_parties_test.csv")
external_parties_train = pd.read_csv("external_parties_train.csv")

Delete internal transaction

In [16]:
booking = account_booking_test
parties = external_parties_test

In [17]:
df_parties = pd.DataFrame(parties)


df_booking = pd.DataFrame(booking).drop_duplicates(subset=["transaction_reference_id"], keep=False)
#print(f"reduction {1-len(df)/len(account_booking_test)}")

In [18]:
def compare(inp: pd.DataFrame, truth: pd.DataFrame):
    assert len(inp) == len(truth)
    tp = 0
    fp = 0
    fn = 0
    
    truth_map = {}
    guess_map = {}
    for _, x in truth.iterrows():
        truth_map[x["transaction_reference_id"]] = x["external_id"]
    
    for _, x in inp.iterrows():
        guess_map[x["transaction_reference_id"]] = x["external_id"]

    inp = inp[inp.duplicated(subset='external_id', keep=False)]
    truth = truth[truth.duplicated(subset='external_id', keep=False)]

    truth_gr = truth.groupby("external_id")["transaction_reference_id"].apply(list)
    guess_gr = inp.groupby("external_id")["transaction_reference_id"].apply(list)

    
    for t_group in truth_gr:
        for a in t_group:
            for b in t_group:
                if a == b:
                    continue

                ag, bg = guess_map[a], guess_map[b]

                if ag == bg:
                    tp += 1
                else:
                    fn += 1

    for g_group in guess_gr:
        for a in g_group:
            for b in g_group:
                if a == b:
                    continue
                
                at, bt = truth_map[a], guess_map[b]

                if at != bt:
                    fp += 1
            

    # for i,a in inp.iterrows():
    #     for j,b in inp.iterrows():
           
    #         if a["transaction_reference_id"] == b["transaction_reference_id"]:
    #             continue
            
    #         a_val, b_val = a["external_id"], b["external_id"]
            
    #         at = truth_map[a["transaction_reference_id"]]
    #         bt = truth_map[b["transaction_reference_id"]]

    #         atg, btg = at["external_id"], bt["external_id"]

    #         if (a_val == b_val) and (atg == btg):
    #             tp += 1
            
    #         if (a_val == b_val) and (atg != btg):
    #             fp += 1
            
    #         if (a_val != b_val) and (atg == btg):
    #             fn += 1
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
            
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1
            

                

print(compare(df_parties, df_parties))

KeyError: 'external_id'